In [5]:
import pymongo as mg
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns


# return a mongodb client
def get_client(usr, pwd, address, port='27017'):
    url = 'mongodb://%s:%s@%s:%s/RESTfulSwarmDB' % (usr, pwd, address, port)
    return mg.MongoClient(url)


# return a database object
def get_db(client, db_name):
    return client[db_name]


# return a collection cursor object
def get_col(db, col_name):
    return db[col_name]


# query data in a job collection
def query_col(col):
    return list(col.find({}))[0]


# get a list of job collection name
def get_all_cols(db):
     return db.collection_names()

# delete a collection(a job in my work)
def drop_col(client, db_name, col_name):
    db = client[db_name]
    col = db[col_name]
    col.drop()

def find_col(col):
    return list(col.find({}))

def update_doc(col, filter_key, filter_value, target_key, target_value):
    return col.update_one({filter_key: filter_value}, {"$set": {target_key: target_value}})

In [6]:
client = get_client('admin', 'kzw', '129.114.108.18')
db = get_db(client, 'RESTfulSwarmDB')

In [7]:
def clear_db():
    all_cols = get_all_cols(db)
    if 'WorkersResourceInfo' in all_cols:
        # Drop worker resource info collection
        drop_col(client, 'RESTfulSwarmDB', 'WorkersResourceInfo')

    if 'WorkersInfo' in all_cols:
        # Reset worker info collection
        workers_info_col = get_col(db, 'WorkersInfo')
        workers_info_data = find_col(workers_info_col)
        for index, worker in enumerate(workers_info_data[:]):
            for cpu in worker['CPUs']:
                workers_info_data[index]['CPUs'][cpu] = False
            update_doc(col=workers_info_col,
                          filter_key='hostname',
                          filter_value=worker['hostname'],
                          target_key='CPUs',
                          target_value=workers_info_data[index]['CPUs'])
    print('Reset MongoDB.')
clear_db()

Reset MongoDB.


In [8]:
for col in get_all_cols(db):
    db[col].drop()